In [1]:
import scipy
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [2]:
df = pd.read_csv("Data.csv", sep=";")
df.dtypes

Direksi Perempuan    int64
Direksi Laki-Laki    int64
Harga penutupan      int64
Total Shares         int64
Total Utang          int64
Total Asset          int64
dtype: object

In [3]:
X = (df['Direksi Perempuan'])/(df['Direksi Perempuan'] + df['Direksi Laki-Laki'])
#X = df["Direksi Perempuan"]
nilai_pasar = df["Total Shares"] * df["Harga penutupan"]
np_plus_liability = nilai_pasar + df["Total Utang"]
Y = np_plus_liability / df["Total Asset"]
# Y = nilai_pasar / df["Total Asset"]

data = pd.DataFrame({"X":X, "Y":Y})
X_mean = np.mean(data["X"])
X_std = np.std(data["X"])
Y_mean = np.mean(data["Y"])
Y_std = np.std(data["Y"])

In [4]:
data_sample = pd.DataFrame({"X": X, "Y": Y})
data_sample.describe()

,X,Y
count,140.000000,140.000000
mean,0.179244,1.311243
std,0.150606,2.089952
min,0.000000,0.159993
25%,0.000000,0.928060
50%,0.181818,0.974360
75%,0.255682,1.101783
max,0.750000,18.327231


In [5]:
# data_sample.drop(data_sample[data_sample["X"] == 0].index, inplace=True)
# np.size(np.where(data_sample["X"] == 0))

In [6]:
X_with_constant = sm.add_constant(data_sample["X"])
result = sm.OLS(data_sample["Y"], X_with_constant).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.007
Method:                 Least Squares   F-statistic:                   0.04680
Date:                Sun, 07 May 2023   Prob (F-statistic):              0.829
Time:                        23:34:33   Log-Likelihood:                -301.33
No. Observations:                 140   AIC:                             606.7
Df Residuals:                     138   BIC:                             612.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.3570      0.276      4.915      0.000       0.811       1.903
X             -0.2555      1.181     -0.216      0.829      -2.591       2.080
==============================================================================
Omnibus:                      238.566   Durbin-Watson:                   0.891
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            22227.793
Skew:                           7.713   Prob(JB):                         0.00
Kurtosis:                      62.771   Cond. No.                         6.88
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [7]:
bX = data_sample["X"] * result.params[0]
abs_residual = np.abs(data_sample["Y"] - result.params["const"] - bX)

In [8]:
hetero_result = sm.OLS(abs_residual, X_with_constant).fit()
hetero_result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.008
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     1.063
Date:                Sun, 07 May 2023   Prob (F-statistic):              0.304
Time:                        23:34:33   Log-Likelihood:                -289.48
No. Observations:                 140   AIC:                             583.0
Df Residuals:                     138   BIC:                             588.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.6875      0.254      2.710      0.008       0.186       1.189
X              1.1188      1.085      1.031      0.304      -1.027       3.265
==============================================================================
Omnibus:                      243.522   Durbin-Watson:                   0.984
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            24516.365
Skew:                           7.987   Prob(JB):                         0.00
Kurtosis:                      65.830   Cond. No.                         6.88
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [9]:
XQ3,XQ1 = np.percentile(data_sample["X"], [75 ,25])
XIQR = XQ3 - XQ1 

In [10]:
lower_X = XQ1 - (1.5 * XIQR)
upper_X = XQ3 + (1.5 * XIQR)

In [11]:
data_sample["X"][(data_sample["X"] < lower_X) | (data_sample["X"] > upper_X)]

56    0.75
Name: X, dtype: float64

In [12]:
data_sample.drop(data_sample["X"][(data_sample["X"] < lower_X) | (data_sample["X"] > upper_X)].index, inplace=True)

In [13]:
YQ3,YQ1 = np.percentile(data_sample['Y'], [75 ,25])
YIQR = YQ3 - YQ1 

In [14]:
lower_Y = YQ1 - (1.5 * YIQR)
upper_Y = YQ3 + (1.5 * YIQR)

In [15]:
data_sample.drop(data_sample["Y"][(data_sample["Y"] < lower_Y) | (data_sample["Y"] > upper_Y)].index, inplace=True)

In [16]:
data_sample.describe()

,X,Y
count,116.000000,116.000000
mean,0.191473,1.002042
std,0.141011,0.118480
min,0.000000,0.688744
25%,0.090909,0.932092
50%,0.200000,0.970017
75%,0.272727,1.056289
max,0.545455,1.385464


In [17]:
X_with_constant = sm.add_constant(data_sample["X"])
result = sm.OLS(data_sample["Y"], X_with_constant).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.015
Model:                            OLS   Adj. R-squared:                  0.007
Method:                 Least Squares   F-statistic:                     1.762
Date:                Sun, 07 May 2023   Prob (F-statistic):              0.187
Time:                        23:34:33   Log-Likelihood:                 84.224
No. Observations:                 116   AIC:                            -164.4
Df Residuals:                     114   BIC:                            -158.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.9822      0.019     52.972      0.000       0.945       1.019
X              0.1037      0.078      1.327      0.187      -0.051       0.258
==============================================================================
Omnibus:                       20.565   Durbin-Watson:                   1.020
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               25.068
Skew:                           1.035   Prob(JB):                     3.60e-06
Kurtosis:                       3.950   Cond. No.                         7.39
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [18]:
bX = data_sample["X"] * result.params["X"]

In [19]:
# Y = a + bX + e
# e = Y - a - bX

abs_residual = np.abs(data_sample["Y"] - result.params["const"] - bX)

In [20]:
hetero_result = sm.OLS(abs_residual, X_with_constant).fit()
hetero_result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.009
Method:                 Least Squares   F-statistic:                  0.001017
Date:                Sun, 07 May 2023   Prob (F-statistic):              0.975
Time:                        23:34:33   Log-Likelihood:                 138.43
No. Observations:                 116   AIC:                            -272.9
Df Residuals:                     114   BIC:                            -267.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0909      0.012      7.825      0.000       0.068       0.114
X              0.0016      0.049      0.032      0.975      -0.095       0.099
==============================================================================
Omnibus:                       44.528   Durbin-Watson:                   1.336
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               92.279
Skew:                           1.609   Prob(JB):                     9.16e-21
Kurtosis:                       5.956   Cond. No.                         7.39
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""